In [1]:
import ollama
import pdfplumber


### [!] Show modeling in here

In [2]:
!ollama list

NAME               ID              SIZE      MODIFIED    
llama3.2:latest    a80c4f17acd5    2.0 GB    5 hours ago    


In [3]:
prompt = "เงินประกันคืออะไร"
res = ollama.chat(model="llama3.2"
                  ,messages=[{"role":"user","content":prompt}])
res["message"]["content"]

'เงินประกันเป็นหนึ่งในส่วนประกอบสำคัญของการลงทุน ซึ่งเป็นการลงทุนในหุ้นหรือหลักทรัพย์อื่นๆ โดยมีการรับประกันว่าจะได้รับรายได้จากการลงทุน โดยทั่วไป เงินประกันจะถูกกำหนดไว้ขึ้นล่วงหน้า และจะได้รับการชำระเงินโดยมีจำนวนการชำระเท่ากันกับส่วนแบ่งสิทธิในหลักทรัพย์ที่เป็นรูปธรรม'

In [4]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:/Program Files (x86)/Tesseract-OCR/tesseract.exe"


In [5]:
import pdf2image
text = pytesseract.image_to_string(image='Files/Testing/Image/abstract.jpg')

path = "Files/Testing/pdf/2410.08406v1.pdf"

try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract


def pdf_to_img(pdf_file):
    return pdf2image.convert_from_path(pdf_file)


def ocr_core(file):
    text = pytesseract.image_to_string(file)
    return text


def print_pages(pdf_file):
    images = pdf_to_img(pdf_file)
    for pg, img in enumerate(images):
        print(ocr_core(img))


print_pages(path)


#text

:24l0.08406vl [CS.HC] 10 Oct 2024

a1‘Xiv

Promptly Yours? A Human Subject Study on
Prompt Inference in AI-Generated Art

Khoi Trinhl, Joseph Spracklenz, Raveen Wijewickramaz,
Bimal Viswanath3, Murtuza Jadliwalaz, Anindya Maiti‘
khoitrinh @ 0u.edu, joseph. spracklen @my. utsa.edu, raveen. wtjewickrama @ utra. edu,

vbimal @ vzﬁedu, murtuzajadliwala @ utsa. edu, am @014. edu

1 University of Oklahoma
2 University of Texas at San Antonio
3 Virginia Tech

Abstract

The emerging ﬁeld of AI-generated art has witnessed the rise
of prompt marketplaces, where creators can purchase, sell,
or share prompts to generate unique artworks. These market-
places often assert ownership over prompts, claiming them
as intellectual property. This paper investigates whether con-
cealed prompts sold on prompt marketplaces can be consid-
ered as secure intellectual property, given that humans and AI
tools may be able to approximately infer the prompts based
on publicly advertised sample images accompanying ea

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

file_path = 'Files/Testing/pdf/medium.pdf'

loader = PyPDFLoader(file_path=file_path)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

split_docs = splitter.split_documents(documents)



In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# สร้าง embedding model (ไม่ใช้ OpenAI ก็ได้)
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory="medium_db"  # โฟลเดอร์เก็บข้อมูล
)

vectordb.persist()

C:\Users\markereversey\AppData\Local\Temp\ipykernel_16960\1812023079.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [16]:
vectordb = Chroma(
    persist_directory="medium_db",
    embedding_function=embedding
)

query = "บทความนี้เกี่ยวกับอะไร"
results = vectordb.similarity_search(query, k=5)

#for doc in results:
#    print(f"[หน้า: {doc.metadata.get('page', '?')}]")
#    print(doc.page_content)
#    print("-" * 30)


In [17]:
vectordb = Chroma(
    persist_directory="medium_db",
    embedding_function=embedding
)

def generate_res(query):
    results = vectordb.similarity_search(query, k=5)

    context = "\n".join([doc.page_content for doc in results])

    prompt = f"Answer the question based on the following context:\n{context}\n\nQuestion"

    response = ollama.chat(model="llama3.2", messages=[
        {"role":"system", "content":"You are an assistant."},
        {"role":"user", "content": prompt}
    ])

    return response["message"]["content"]
generate_res("standardization คืออะไร")

'การ Standardization เป็นขั้นตอนแรกในการท างานของ Principal Component Analysis (PCA) และเป็นสิ่งสำคัญอย่างยิ่งต่อกระบวนการนี้ การทำตามขั้นตอนนี้ช่วยให้แต่ละตัวแปรใน Dataset มี Range ที่เท่ากัน ซึ่งช่วยป้องกันผลกระทบของตัวแปรที่มี Range กว้างกว่า และทำให้การวิเคราะห์ง่ายขึ้น'